In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

def match_contours(red_contours_info, green_contours_info):
    matches = []
    used_red_contours = set()
    used_green_contours = set()

    for red_contour in red_contours_info:
        # Verificar si este contorno rojo ya está emparejado
        if id(red_contour) in used_red_contours:
            continue

        best_match = None
        best_angle_match = float('inf')  # Para encontrar el mejor ángulo
        best_score = float('inf')       # Para desempatar usando área y perímetro

        for green_contour in green_contours_info:
            # Verificar si este contorno verde ya está emparejado
            if id(green_contour) in used_green_contours:
                continue

            # Calcular distancia entre centros
            center_distance = np.linalg.norm(
                np.array(red_contour['center']) - np.array(green_contour['center'])
            )

            # Descartar si la distancia excede 3.5 veces la anchura del contorno rojo
            if center_distance > 3.2 * red_contour['width']:
                continue

            # Calcular diferencia de ángulo
            angle_diff = abs(red_contour['angle'] - green_contour['angle'])

            # Descartar si la diferencia de ángulo es mayor a 10 grados
            # if angle_diff > 20:
            #     continue

            # Calcular diferencias adicionales para desempatar
            area_diff = abs(red_contour['area'] - green_contour['area'])
            perimeter_diff = abs(red_contour['perimeter'] - green_contour['perimeter'])

            # Crear puntaje de desempate
            score = area_diff + perimeter_diff

            # Actualizar el mejor match
            if angle_diff < best_angle_match or (angle_diff == best_angle_match and score < best_score):
                best_angle_match = angle_diff
                best_score = score
                best_match = green_contour

        if best_match:
            matches.append((red_contour, best_match))
            used_red_contours.add(id(red_contour))
            used_green_contours.add(id(best_match))

    return matches

def cut_bounding_box(matched_contours, image):
    extracted_images = []

    for red_contour, green_contour in matched_contours:
        # Obtener los puntos de los contornos
        red_points = np.array(red_contour['corners'])
        green_points = np.array(green_contour['corners'])

        # Combinar los puntos de ambos contornos
        all_points = np.vstack((red_points, green_points))

        # Calcular la bounding box que contenga todos los puntos
        x_min, y_min = np.min(all_points, axis=0)
        x_max, y_max = np.max(all_points, axis=0)

        # Convertir los centros a tuplas de enteros
        red_center = tuple(map(int, red_contour['center']))
        green_center = tuple(map(int, green_contour['center']))

        # # Dibujar la línea que une los centros en la imagen original
        # cv2.line(image, red_center, green_center, (255, 0, 0), 2)

        # Calcular el ángulo entre los centros
        x0, y0 = red_contour['center']
        x1, y1 = green_contour['center']
        angle = np.arctan2(y1 - y0, x1 - x0) * 180 / np.pi

        # Rotar la imagen original para que la línea entre los centros sea horizontal
        M = cv2.getRotationMatrix2D(red_center, angle, 1)
        rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
        
        # Transformar las coordenadas de los contornos según la rotación
        ones = np.ones((red_points.shape[0], 1))
        red_points_homogeneous = np.hstack([red_points, ones])
        transformed_red_points = (M @ red_points_homogeneous.T).T

        green_points_homogeneous = np.hstack([green_points, ones])
        transformed_green_points = (M @ green_points_homogeneous.T).T

        # Combinar los puntos transformados
        transformed_all_points = np.vstack((transformed_red_points, transformed_green_points))

        # Calcular la nueva bounding box en la imagen rotada
        x_min, y_min = np.min(transformed_all_points, axis=0).astype(int)
        x_max, y_max = np.max(transformed_all_points, axis=0).astype(int)

        # Validar las coordenadas para asegurarse de que estén dentro de los límites
        h, w = rotated_image.shape[:2]
        x_min, x_max = max(0, x_min), min(w, x_max)
        y_min, y_max = max(0, y_min), min(h, y_max)

        # Verificar que las dimensiones del recorte sean válidas
        if x_min < x_max and y_min < y_max:
            cropped_image = rotated_image[y_min:y_max, x_min:x_max]

            if cropped_image.size > 0:  # Comprobar que el recorte no sea vacío
                extracted_images.append(cropped_image)
            else:
                print(f"Warning: Empty crop for contours {red_contour['center']} and {green_contour['center']}")
        else:
            print(f"Warning: Invalid crop coordinates for contours {red_contour['center']} and {green_contour['center']}")

    return extracted_images

def process_image(image_path):
    # Cargar la imagen
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Desenfoque
    image_blurred = cv2.GaussianBlur(image, (7, 7), 0)

    # Convertir a escala de grises y HSV
    gray_image = cv2.cvtColor(image_blurred, cv2.COLOR_RGB2GRAY)
    hsv_image = cv2.cvtColor(image_blurred, cv2.COLOR_RGB2HSV)

    # Aplicar máscaras roja y verde
    def apply_red_mask(hsv_image):
        lower_red = np.array([125, 20, 0])
        upper_red = np.array([179, 255, 255])
        return cv2.inRange(hsv_image, lower_red, upper_red)

    def apply_green_mask(hsv_image):
        lower_green = np.array([50, 40, 0])
        upper_green = np.array([110, 255, 160])
        return cv2.inRange(hsv_image, lower_green, upper_green)

    mask_red = apply_red_mask(hsv_image)
    mask_green = apply_green_mask(hsv_image)

    # Erosionar y cerrar las máscaras
    def close_erode_mask(mask):
        kernel = np.ones((7, 7), np.uint16)
        mask_closed = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        kernel = np.ones((5, 5), np.uint16)
        return cv2.dilate(mask_closed, kernel, iterations=1)

    mask_red = close_erode_mask(mask_red)
    mask_green = close_erode_mask(mask_green)

    # Segmentar las imágenes usando las máscaras
    def apply_mask_to_image(image, mask):
        return cv2.bitwise_and(image, image, mask=mask)

    segmented_red = apply_mask_to_image(gray_image, mask_red)
    segmented_green = apply_mask_to_image(gray_image, mask_green)

    # Aplicar el filtro Sobel
    def sobel_filter_image(image, kernel_size):
        img_sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel_size)
        img_sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel_size)
        img_sobel = np.sqrt(img_sobel_x**2 + img_sobel_y**2)
        img_sobel = cv2.normalize(img_sobel, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        _, filtered_image = cv2.threshold(img_sobel, 30, 255, cv2.THRESH_BINARY)
        return filtered_image

    filtered_green = sobel_filter_image(segmented_green, 11)
    filtered_red = sobel_filter_image(segmented_red, 11)

    # Encontrar contornos
    def find_contours(filtered_image):
        contours, _ = cv2.findContours(filtered_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filtered_contours = [
            contour for contour in contours
            if cv2.contourArea(contour) > 4000 and 0.5 < cv2.boundingRect(contour)[2] / cv2.boundingRect(contour)[3] < 1.3
        ]
        return filtered_contours

    filtered_green_contours = find_contours(filtered_green)
    filtered_red_contours = find_contours(filtered_red)

    # Extraer información de los contornos
    def extract_contour_info(contours):
        contour_info = []
        for contour in contours:
            area = round(cv2.contourArea(contour), 3)
            x, y, w, h = cv2.boundingRect(contour)
            perimeter = round(cv2.arcLength(contour, True), 3)
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int16(box)
            corners = [(corner[0], corner[1]) for corner in box]
            contour_info.append({
                'corners': corners,
                'center': (round(x + w / 2, 3), round(y + h / 2, 3)),
                'width': round(w, 3),
                'height': round(h, 3),
                'area': area,
                'aspect_ratio': round(w / h, 3) if h != 0 else None,
                'perimeter': perimeter,
                'angle': round(rect[-1], 3)
            })
        return contour_info

    green_contour_info = extract_contour_info(filtered_green_contours)
    red_contour_info = extract_contour_info(filtered_red_contours)

    # Emparejar contornos
    matched_contours = match_contours(red_contour_info, green_contour_info)

    # Cortar imágenes de los contornos emparejados
    extracted_images = cut_bounding_box(matched_contours, image)

    return extracted_images

# Ruta de la carpeta con las imágenes
input_folder = r"Real Images"
output_folder = r"Processed Images"
os.makedirs(output_folder, exist_ok=True)

# Procesar todas las imágenes
for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        print(f"Processing {file_name}...")
        extracted_images = process_image(file_path)
        for i, img in enumerate(extracted_images):
            output_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}_extracted_{i+1}.jpg")
            cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

Processing 0000_1_04_10.png...
Angle between centers: -98.92°
Processing 0000_1_04_4.png...
Processing 0000_1_04_9.png...
Angle between centers: -24.22°
Processing 1103_1_G1_1.jpg...
Angle between centers: 148.19°
Processing 1103_1_G1_2.jpg...
Angle between centers: 70.01°
Processing 1103_1_G1_3.jpg...
Angle between centers: 26.36°
Processing 1103_1_G1_4.jpg...
Angle between centers: 28.27°
Processing 1103_1_G2_00.jpg...
Angle between centers: -3.45°
Processing 1103_1_G2_01.jpg...
Angle between centers: 93.45°
Processing 1103_1_G2_02.jpg...
Angle between centers: -174.94°
Processing 1103_1_G2_03.jpg...
Angle between centers: -81.26°
Processing 1103_1_G2_04.jpg...
Processing 1111_1_04_10.png...
Processing 1111_1_04_11.png...
Angle between centers: 3.95°
Processing 1111_1_04_12.png...
Angle between centers: 26.96°
Processing 1111_1_04_13.png...
Angle between centers: -103.44°
Processing 1234_1_04_1.png...
Angle between centers: 173.89°
Processing 1234_1_04_2.png...
Angle between centers: